In [1]:
import torch
from binhd.embeddings import CategoricalEncoder
from binhd.classifiers import BinHD
from module.model import EncodersModel


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [3]:
from datasets.soybean import Soybean


dataset = Soybean()
dimension = 1000
num_levels = 100
batch_size = 1000
low = 0
high = num_levels
oper = "bind"

In [4]:
import numpy as np
from sklearn.calibration import LabelEncoder

print(dataset.features)

X = dataset.features
categorical_features = X[dataset.categorical_features]
num_categories_per_feature = [categorical_features[col].nunique() for col in categorical_features.columns]
categorical_encoder = CategoricalEncoder(dimension)
X[dataset.categorical_features] = categorical_encoder.fit_transform(categorical_features)


# Substituir '?' por NaN
X = X.replace('?', np.nan)

# Preencher valores ausentes com o valor mais frequente de cada coluna (moda)
for col in X.columns:
    X[col] = X[col].fillna(X[col].mode()[0])


y = dataset.targets
le = LabelEncoder()
y_encoded = torch.tensor(le.fit_transform(y))

     hair  feathers  eggs  milk  airborne  aquatic  predator  toothed  \
0       1         0     0     1         0        0         1        1   
1       1         0     0     1         0        0         0        1   
2       0         0     1     0         0        1         1        1   
3       1         0     0     1         0        0         1        1   
4       1         0     0     1         0        0         1        1   
..    ...       ...   ...   ...       ...      ...       ...      ...   
96      1         0     0     1         0        0         0        1   
97      1         0     1     0         1        0         0        0   
98      1         0     0     1         0        0         1        1   
99      0         0     1     0         0        0         0        0   
100     0         1     1     0         1        0         0        0   

     backbone  breathes  venomous  fins  legs  tail  domestic  catsize  
0           1         1         0     0     4     

/home/marinapiragibe/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
model = BinHD(dimension, dataset.num_classes)

encorders_model = EncodersModel(model)
encorders_model.start_encoders(X, dimension, num_levels, low, high)

y_encoded = torch.tensor(y_encoded).to(device)
encorders_model.run_encoders(X, y_encoded, device, oper)
encorders_model.split_train_test()
encorders_model.run_all_types()

hair        int32
feathers    int32
eggs        int32
milk        int32
airborne    int32
aquatic     int32
predator    int32
toothed     int32
backbone    int32
breathes    int32
venomous    int32
fins        int32
legs        int32
tail        int32
domestic    int32
catsize     int32
dtype: object
BinHD Record Encoder: Accuracy =  0.2903225806451613
BinHD Ngram Encoder: Accuracy =  0.9354838709677419


/tmp/ipykernel_12342/1936118792.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_encoded = torch.tensor(y_encoded).to(device)
/home/marinapiragibe/ia_verde/trabalho_hdc/module/model.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(y).to(device)
